# Crash Report Jan 18, 2016

Version: 2015050021

In [1]:
"""
General import statements
"""
import crash_analysis
import pandas as pd

In [2]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [2]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = False
cache_location = 'src/data/2015050021_feb7_530pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [7]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)

# 5350 Crashes

print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin15.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'MemoryUsageKbytes', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin15.INI', 'ProblemDescription', 'SystemTimeUTC', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml']
2015050021    5350
Name: AppVersion, dtype: int64
5350


In [7]:
from crash_analysis import filter_dataframe
df = filter_dataframe(df, AppVersion=2016070012)
len(df)

6054

In [18]:
parse_last = lambda x: str(x).split('\\')[-1]
df['ExceptionModule'].apply(parse_last).value_counts()

KERNELBASE.dll                         2721
protax15.exe                            753
fpdfsdk.dll                             673
nan                                     451
c4dll.dll                               226
ntdll.dll                               157
mfc110.dll                               62
user32.dll                               53
CrashRpt1403.dll                         36
kernel32.dll                             28
ieframe.dll                              27
TerR.dll                                 27
urlmon.dll                               20
Rep32.dll                                17
PSLogR.DLL                               11
MSMAPI32.DLL                             10
mshtml.dll                                8
MSPST32.DLL                               8
atiumdag.dll                              7
iertutil.dll                              6
MSONSEXT.DLL                              6
msvcr110.dll                              5
mscorlib.ni.dll                 

In [27]:
df[df['ExceptionModule'].fillna(' ').str.contains('msvcr110.dll')]['CrashGUID']

1181    3a0d129d-6e32-4e27-8639-b0241a623e88
3375    9fb86d23-f35c-4b9e-8d5c-edf9a913826c
3997    bdbdba42-a0f3-4cbb-b381-0de3c7818fa5
4728    e3730e6a-03d8-4dc1-91ad-11a6f36e1f25
4824    e7c688da-3d12-44d6-9b8f-f228543b6e02
Name: CrashGUID, dtype: object

In [10]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe



def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType','OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})

    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()


    total = sum(field2_hist)

    print("Query {0} \t count: {1} \t {2} ({3}): {4}".format(n+1, count, field1, total, top_key.encode('ascii', 'ignore')))

    print(field2_hist)
    print('\n')
    
# 
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.height', 5000)
# [print_parings(n, groupings='ProblemDescription') for n in range(5)]
# print_parings(0, groupings='CustNum')
# [print_parings(n, field1='ExceptionAddress') for n in [0]]
# [print_parings(n, field1='ExceptionAddress', groupings='ProblemDescription') for n in [0]]

[print_parings(n, field1='ExceptionModule') for n in range(9)]

# file_counts_df = filter_dataframe(df, ExceptionAddress='0x100425c3')
# print(len(file_counts_df))

# file_counts_df = filtered_df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
# file_counts_df.hist(bins=50)
# # file_counts_df = pd.to_numeric(file_counts_df, errors='ignore')
# # file_counts_df.mean()
# print(file_counts_df.min(), file_counts_df.max())
# df[df['CustNum'] == '0614456913']

AttributeError: 'dict' object has no attribute 'iteritems'

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception